In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mgimg
import matplotlib.colors as colors
import scipy as sp
import numpy as np
import nestkit_modified as nestkit
#from cornerhk import corner
import pickle as pickle
from IPython.display import display
import forwardmodel
import ciamod
import TPmod
import cloudnest
#import band
import brewtools
import settings
from pymultinest.analyse import Analyzer
from astropy.convolution import convolve, convolve_fft
from astropy.convolution import Gaussian1DKernel
from scipy.io.idl import readsav
from scipy import interpolate
from scipy.interpolate import interp1d
from scipy.interpolate import InterpolatedUnivariateSpline
from bensconv import conv_non_uniform_R

In [2]:
import matplotlib as mpl
#import smplotlib
from matplotlib import colors
from matplotlib.ticker import AutoMinorLocator, ScalarFormatter, FuncFormatter, LogLocator, MultipleLocator
import matplotlib.lines as mlines

from tqdm import tqdm
from scipy import integrate, constants

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
object_name = "W0359"

In [5]:
runname ="W0359_5_knots"

#argfile = "/Users/harshil/PycharmProjects/Brewster/Retrieved_results/Nested_sampling_files/W0359/"+runname+"_runargs.pic"
argfile = "/Users/harshil/PycharmProjects/Brewster/Retrieved_results/Nested_sampling_files/W0359/WO_photometry/"+runname+"_runargs.pic"

runargs = brewtools.pickle_load(argfile)

gases_myP,chemeq,dist,dist_err,cloudtype,do_clouds,gasnum,gaslist,cloudnum,inlinetemps,coarsePress,press,inwavenum,linelist,cia,ciatemps,use_disort,fwhm,obspec,proftype,do_fudge, prof,do_bff,bff_raw,ceTgrid,metscale,coscale, wavelength_cutoff, sf, sf_err = runargs

ndim,pnames = nestkit.countdims(runargs, plist=True)

In [6]:
#a = Analyzer(ndim, outputfiles_basename = "/Users/harshil/PycharmProjects/Brewster/Retrieved_results/Nested_sampling_files/W0359/"+runname)
a = Analyzer(ndim, outputfiles_basename = "/Users/harshil/PycharmProjects/Brewster/Retrieved_results/Nested_sampling_files/W0359/WO_photometry/"+runname)


stats = a.get_stats()
bestfit_params = a.get_best_fit()
#print(bestfit_params)

#theta_max = np.array(bestfit_params['parameters'])
#max_like = bestfit_params['log_likelihood']

tmp = a.get_equal_weighted_posterior()
samples = tmp[:,:(tmp.shape[1]-1)]

Tsamples = samples[:,ndim-5:]
nsamps = Tsamples.shape[0]
nlayers = len(press)
Tprofs = np.empty([nlayers,Tsamples.shape[0]])
for i in range(0,nsamps):
    Tprofs[:,i] = TPmod.set_prof(1,coarsePress,press,Tsamples[i,:])

Tlays = np.empty([nlayers,5])
for i in range(0,nlayers):
    junk = Tprofs[i,:]
    junk2 = np.percentile(junk, [2.4,16, 50, 84,97.6],axis=0)
    junk3 = np.array(junk2)
    Tlays[i,:] = junk3[:]

 # get the cloud stuff
cloudsamples = samples[:,ndim-9:ndim-5]

cloud_Top = np.percentile(cloudsamples[:, 0], [16, 50, 84])
cloud_height = np.percentile(cloudsamples[:, 1], [16, 50, 84])

  analysing data from /Users/harshil/PycharmProjects/Brewster/Retrieved_results/Nested_sampling_files/W0359/WO_photometry/W0359_5_knots.txt


In [16]:
#np.savetxt('/Users/harshil/PycharmProjects/Brewster/Retrieved_results/Retrieved_posterior_samples/'+object_name+'_samples.txt', np.c_[samples])

In [17]:
#np.savetxt('/Users/harshil/PycharmProjects/Brewster/Retrieved_results/Retrieved_thermal_profile/'+object_name+'_thermal_profile.txt', np.c_[Tlays])

In [7]:
np.savetxt('/Users/harshil/PycharmProjects/Brewster/Retrieved_results/Retrieved_posterior_samples/'+object_name+'_samples_wo_photometry.txt', np.c_[samples])
np.savetxt('/Users/harshil/PycharmProjects/Brewster/Retrieved_results/Retrieved_thermal_profile/'+object_name+'_thermal_profile_wo_photometry.txt', np.c_[Tlays])

In [ ]:
#np.savetxt(runname+'_VMRs.dat', np.c_[gasVMR[:,0],gasVMR[:,1],gasVMR[:,2]])

In [8]:
# get diagnostics along with the spectrum
gnostics = 0
#shiftspec, photspec, tauspec,cfunc = nestkit.modelspec(theta_max,runargs,gnostics)

In [62]:
#topspec = brewtools.proc_spec(shiftspec,theta_max,fwhm,chemeq,gasnum,obspec)

In [ ]:
"""topspec = brewtools.proc_spec(shiftspec,theta_max,fwhm,chemeq,gasnum,obspec)
#print(topspec.shape)
pltspec = np.zeros((500,obspec[0,:].size))

samp= np.empty(ndim)
sid = np.zeros(500)
for i in range (0,500):
    sid[i]= np.random.randint(0,high = len(samples))
    samp = samples[int(sid[i]),:]
    shiftspec, photspec, tauspec,cfunc = nestkit.modelspec(samp,runargs,gnostics)
    pltspec[i,:] = brewtools.proc_spec(shiftspec,samp,fwhm,chemeq,gasnum,obspec)

specdist = np.empty([obspec[0].size,5])
for i in range(0,obspec[0].size):
    junk = pltspec[:,i]
    junk2 = np.percentile(junk, [2.4,16, 50, 84,97.6],axis=0)
    junk3 = np.array(junk2)
    specdist[i,:] = junk3[:]
"""

In [9]:
def proc_spec(shiftspec,theta,fwhm,chemeq,gasnum,obspec):
    import numpy as np
    import scipy as sp
    from bensconv import conv_non_uniform_R

    if chemeq == 0:
        if (gasnum[gasnum.size-1] == 21):
            ng = gasnum.size - 1
        elif (gasnum[gasnum.size-1] == 23):
            ng = gasnum.size -2
        else:
            ng = gasnum.size
            invmr = theta[0:ng]

    else:
        ng = 2

    modspec = np.array([shiftspec[0,::-1],shiftspec[1,::-1]])

     # Modified by Harshil
    if (fwhm == 999):
        sf_NIRSpec = theta[ng + 4]
        sf_MIRI = theta[ng + 5]

        mr_NIRSpec = np.where(modspec[0, :] < wavelength_cutoff)[0]
        or_NIRSpec = np.where(obspec[0, :] < wavelength_cutoff)[0]

        mr_MIRI = np.where(np.logical_and(modspec[0, :] > wavelength_cutoff, modspec[0, :] < 13.0))[0]
        or_MIRI = np.where(np.logical_and(obspec[0, :] > wavelength_cutoff, obspec[0, :] < 13.0))[0]

        mr_photometry = np.where(modspec[0, :] > 13)[0]
        or_photometry = np.where(obspec[0, :] > 13)[0]

        R = obspec[-1, :]

        NIRSpec = sf_NIRSpec * conv_non_uniform_R(obspec[:, or_NIRSpec], modspec[:, mr_NIRSpec], R[or_NIRSpec])
        MIRI = sf_MIRI * conv_non_uniform_R(obspec[:, or_MIRI], modspec[:, mr_MIRI], R[or_MIRI])
        photometry = conv_non_uniform_R(obspec[:, or_photometry], modspec[:, mr_photometry], R[or_photometry])

        outspec = np.array(np.concatenate((NIRSpec,MIRI,photometry),axis=0))
        """

    if (fwhm == 999):
        # this is a non-uniform resolving power R.
        R = obspec[-1, :]
        outspec = conv_non_uniform_R(obspec,modspec, R)"""

    return outspec

In [10]:
#topspec = brewtools.proc_spec(shiftspec, theta_max, fwhm, chemeq, gasnum, obspec)
#print(topspec.shape)
pltspec = np.zeros((samples[:, 0].size, obspec[0, :].size))

for i in tqdm(range(0, samples[:, 0].size)):
    samp = samples[i, :]
    shiftspec, photspec, tauspec, cfunc = nestkit.modelspec(samp, runargs, gnostics)
    pltspec[i, :] = proc_spec(shiftspec, samp, fwhm, chemeq, gasnum, obspec)

100%|██████████| 5230/5230 [2:14:35<00:00,  1.54s/it]  


In [11]:
specdist = np.empty([obspec[0].size, 5])
for i in tqdm(range(0, obspec[0].size)):
    junk = pltspec[:, i]
    junk2 = np.percentile(junk, [2.4, 16, 50, 84, 97.6], axis=0)
    junk3 = np.array(junk2)
    specdist[i, :] = junk3[:]

100%|██████████| 580/580 [00:00<00:00, 4693.84it/s]


In [12]:
np.savetxt("/Users/harshil/PycharmProjects/Brewster/Retrieved_results/Retrieved_model_spectrum/0359_model_spectrum_wo_photometry.txt", [specdist[:,2], specdist[:,1],specdist[:,3],specdist[:,0],specdist[:,4]])